### This estimator estimates the security levels according to the Core-SVP estimator (the estimation is according to the propsed medels; sieving is being used here to provide the estimation)

### The file gives the results for both NTRU and DiTRU parameters

In [3]:
import numpy as np
from scipy.special import erf, erfc
from mpmath import mp, mpf, erf, erfc as mperfc
from scipy.special import erfcinv
import numpy as np
mp.dps = 120

In [4]:
from decimal import Decimal
from gmpy2 import mpfr

In [5]:
precision= 120

In [6]:
def sigma_square_real(dr, df, dg, dm,N, cyclic=1, old=0 ):
    """
    Input: - dr,df,dg,and dm: the parameters of the NTRU(in the case of the dihedral group, they are the parameters divided by 2)
           - N: the order for the cyclic group and half the order for the dihedral group
           - cyclic: indicates if the cyclic group is used or not
           - old: 1 means the old NTRU design mentioned in Hoeffistion book otherwise (3f+1) design
           
    
    Output: the value of sigma^2 to be used in the decryption failure
    
    In this function, the value of dr, df, dg, dm can be anything. We have no constraints
    """
    m = 1
    if(cyclic==0): 
        m =2 
    t = 1
    if old ==1:
        t = 9
    return m*(4*(t*dr*dg+df*dm)/(N))

In [7]:
def sigma_square(dr, df,N, cyclic=1, old=0):
    """
    Input:  - N the order of the group
            - dr: parameter for which  r in T(dr,dr)
            - df: parameter for which f in T(df, df)
            - d is half the original d in the case of the dihedral group
            - in the case of the dihedral group, the parameters df/2, ...etc
            - cyclic: refers if the cyclic group is used
            - old: 1 means the old NTRU design mentioned in Hoeffistion book otherwise (3f+1) design
            - In this function, we assum here that 1, -1, 0 in the message m and in g are divided into three halfs.(In other words, we assum that dm=dg= N/3)
    
    Output: the value of sigma^2 to be used in the decryption failure         
    """
    m = 1
    if(cyclic==0): 
        m =2 
    t = 1
    if old ==1:
        t = 9
    return m*(4*(t*dr+df)/(3))

In [8]:
def decryption_failure(N, c, sigma, cyclic=1):
    """
    Input:  - N the order of the cylcic and half the order for the dihedral
            - c the threshold for which you want to calculate the decryption failure
            - sigma: calculated with respect to N,df, dr
            - cyclic: to identify if the cyclic group is used or not
    
    Output: the decryption failure
    """
    n = N
    if cyclic ==0:
        n=2*N
    #print("c =  ",float(c/(sigma*sqrt(2*N))))
    print("n: ", n, "sigma: ", sigma, "c: ", c)
    return n*mperfc(c/(sigma*sqrt(2)))

In [9]:
def get_q_no_error_prime(d,p,old=1):
    """
    Input:  -d: defines from which space T(d,d), we are sampling 
    Output: -primeq: the value of q that makes the decryption successful and prime 
    """
#     value = p*(6*d+1)
    if old:
        value=4*p*d+4*d
    else: ###assuming the old construction that F=pf+1
        value= 8*p*d+2
    print(value)
    q = next_prime(value)
    return q

In [10]:
def get_d_no_error(q,p,old=1):
    """
    Input: -q,p: modulos used in NTRU
           -old: indicates the construction used for NTRU if 1 we are using the old construction h=f^-1*g otherwise h = (pf+1)^-1*g 
    Given q find d that gives no decryption failure
    """
    
    if old:
        d = int(q/(4*p+4))-1 ###q/16-1 for p=3
    else:
        d =int((q-2)/(8*p))
        
    return d

In [11]:
def get_q_no_error(d, old=1, p=3):
    """
    Input: -d: define from which space T(d,d), we are sampling f,g,r
    
    Output: the binary value of q that gives no decryption failure
    """
    if old:
        value=4*p*d+4*d
    else:
        value= 8*p*d+2
    q= 2**(len(bin(value))-2)
    return q


In [12]:
def get_with_2(num):
    """
    Input: num which is a float number refering to the decryption failure probability as a decimal point
    Output: the decryption failure as a power of two 2^-x
    """
    for i in range(900):
        if(2^(-i)<num):
            
            print("Decryption error less than: 2^-{}".format(i))
            return 2^(-i)

In [13]:
def get_sigma_for_decryption_failure(N, dec_failure, q, cyclic, old=0,p=3):
    """
    Input:
           - N: the order of the group for the cyclic group and half the order for the dihedral group
           - dec_failure: the probability of decryption failure
           - p: the modulo p value: often 3
           - q: the value of q
           - cyclic: indicates if the underlying group is cyclic or not
           - old: 1 means the old NTRU design mentioned in Hoeffistion book otherwise (3f+1) design
    Output: the associated value of sigma related to some decryption failure
            
    """
    if old==1:
        c = int(q/2)
    else:
        c = int((q-2)/(2*p))
        #print("c here: ", c)
        
    if cyclic ==1:
        n = N
    else:
        n = 2*N
    #print("dec_failure: ", dec_failure)
    #print("n: ", n)
    #print("c: ", c)
    sigma = c/(sqrt(2)*erfcinv(dec_failure/n))
    
    return float(sigma)

In [14]:
def get_d_with_decryption_failure(sigma, cyclic=1, old=0):
    """
    Input: 
          
           - sigma: sigma that acheives the required levels of security
           - cyclic: indicates the underlying group is cyclic
           - old: 1 means the old NTRU design mentioned in Hoeffistion book otherwise (3f+1) design
    Output: the maximum value of d possible(in the case of the dihedral returned d is half the original d for which f should be sampled)
           
    """
    
    ss = sigma^2 
    
    m = 1
    if(cyclic==0): 
        m =2 
    t = 1
    if old ==1:
        t = 9
        
    d = math.floor((3*ss)/(m*(4*(t+1))) )   
    
    return d

In [15]:
def get_key_norm(N, cyclic=1):
    
    """
    Input: - N: the order of the cylic group and half the order for the dihedral group
             (d = N/3 is calculated based on the value of N)
           - cyclic: refers if the underlying group is cylic or dihedral; defaul cyclic=1
    Output: the norm of the key sqrt(4d+1)
    
    Disclaimer: the norm of the key her is calculated assuming that d = N/3
    """
    
    d = int(N/3)
    mul =1
    if cyclic==0:
        mul = np.sqrt(2) ##estimation on the image of the private key when we are applying one layer of Gentr'y attack
    return float(mul*sqrt(4*d+1))

In [16]:
def get_q(error_rate, sigma, N, old=0, binary=1, p=3, cyclic=1):
    """
    Input: - error rate: the error rate.
           - sigma: the sigma for cyclic or dihedral perspectively.
           - N: the order of the group and half the order for the dihedral: make sure for dihedral it's 2N.
           - old: either old design or new design (for old design we refer to h=gf, for new one we mean h =(3f+1)*g )
           - bin: if 1 means returns the first power of two that satsify the required decrytption failure  otherwise returns q itself.
    
    Outpt: - q that gives the required error rate       
    
    """
    n = N
    if cyclic ==0:
        n = 2*N
    
    q = np.ceil(erfcinv(float(error_rate/n))*sigma*sqrt(2))##c 
    q = q*2
    if old==0:
        q = q*p+2
    if binary:
        q = 2**(len(bin(q))-2) ## returns the binary value.
    return q

In [17]:
def get_q_d_without_error(N, binary=1, p=3, cyclic=1,old=1):
    """
    Twisted dihedral group is treated as a cyclic group
    Input: -N: the order of the cyclic group and half the order for the dihedral group.
           -cyclic: if 1 indicates a cyclic group otherwise indicates a dhideral group.
           -old: indicating which construction is used for NTRU.
    Output: (q, d, beta)
    """
    if cyclic==0:
        n = 2*N
    else:
        n = N
    d = int(n/3)
    print("d: ", d)
    if binary:
        q = get_q_no_error(d,p)
    else:
        q = get_q_no_error_prime(d,p)
    print("q: ", q) 
    beta = get_beta_2016_estimator(N, d, q, p,cyclic)
    qprime = int(q/2)
    if binary==0:
        qprime = next_prime(qprime)
    
    d_prime = get_d_no_error(qprime,p,old)
    print("qprime: ", qprime)
    print("dprime: ", d_prime)
#     print("d_prime:",d_prime)
    beta_prime = get_beta_2016_estimator(N, d_prime, qprime, p,cyclic)
    if beta_prime>=beta:
        q= qprime
        d = d_prime
        beta = beta_prime

    return (q, d, beta)

In [18]:
def get_q_d(error_rate, sigma, N, old=0, binary=1, p=3, cyclic=1):
    """
    Input: - error rate: the error rate.
           - sigma: the sigma for cyclic or dihedral perspectively.
           - N: the order of the group and half the order for the dihedral: make sure for dihedral it's 2N.
           - old: either old design or new design (for old design we refer to h=gf, for new one we mean h =(3f+1)*g )
           - bin: if 1 means returns the first power of two that satsify the required decrytption failure  otherwise returns q itself.
    
    Outpt: - q,d that gives the required error rate while d doesn't need necessirly to be N/3     
    """
    beta_prev = 0
    d_initial = int(N/3)
    n = N
    if cyclic==0:
        n = 2*N
    
    prime_q = np.ceil(erfcinv(float(error_rate/n))*sigma*sqrt(2))##c 
    prime_q = prime_q*2
    if old==0:
        prime_q = prime_q*p+2
    if binary:
        binary_q = 2**(len(bin(prime_q))-2) ## returns the binary value.
        beta_initial = get_beta_2016_estimator(N, d_initial, binary_q, p,cyclic)
        print("beta initial before changing beta: {}".format(beta_initial))
        prev_q = int(binary_q/2)
        if (prime_q-prev_q)<=(binary_q-prime_q):
            sigma = get_sigma_for_decryption_failure(N, error_rate, prev_q, cyclic, old,p=3)##get_sigma for the previous q
            d = get_d_with_decryption_failure(sigma, cyclic, old)##get_d corresponding to the previous q (this d is half d for the dihedral group)
            d = min(int(N/3),d)
            print("d after reducing q by two: ", d)
            beta_prev = get_beta_2016_estimator(N, d, prev_q, p, cyclic)
            print("beta prev after reducing q by two: {}".format(beta_prev))
            #             return (prev_q, max(d_initial,d))
            #         else:
            #             return (binary_q, d_initial) ##otherwise
        
        if beta_initial>beta_prev:
            #beta = beta_initial
            return (binary_q, d_initial, beta_initial)
        else:
            #beta = beta_prev
            return (prev_q, d, beta_prev)
    else:
        beta_prev = get_beta_2016_estimator(N, d_initial, prime_q, p,cyclic) ##for the prime q
    
    return (prime_q, d_initial, beta_prev) ##not binary return prime_q with d
        

## Beta estimation

In [19]:
def left_side(beta, d, norm):
    ## input: beta is the block size
    ## d: the lattice dimension
    ## norm: the secret key norm
    return np.sqrt(beta/d)*norm

In [20]:
def right_side(beta, d, detmn):
    
    ##input: beta : the block size
    ## d: the lattice dimension
    ## detmn: the detmn of the lattice
    
    delta = get_delta_beta(beta)
    t= delta^(2*beta-d-1)*detmn^(1/d)
    
    return t

In [21]:
def get_qary_det(q,n):
    #input n: half the dimension of the q-arry lattice
    return q^(n)

In [22]:
def get_private_key_norm(df,dg,cyclic=1):
    """
    Input: - df the parameter that define the space for which f is sampled T(df,df).
           - dg the parameter that define the space for which g is sampled T(dg,dg).
           - df, dg: are half the orginal for the dihedral group.
           - cyclic refers if the cyclic group is used
    Output: the norm of the key
    """
    norm= np.sqrt(2*df+2*dg+1)
    mul =1
    if cyclic==0:
        mul=np.sqrt(2)
    return (norm*mul)
    

In [23]:
def get_delta_beta(beta):
    """
    Input:  - beta
    Output: - delta_beta  to be used in the 2016 estimation
    """
    t= ((beta/(2*np.pi*np.e))*(np.pi*beta)^(1/beta))^(1/(2*(beta-1)))
    return t

In [24]:
def Binary_search(left_side, right_side, low_beta,high_beta, detmn, norm,dim):
    """
    Binary search function to locate the beta that satisfy 2016-estimator equation
    Input: left_side: the function that evaluates the left side term
           right_side: the function that evaluates the right side term
           low_beta: the lowest value of beta
           high_beta: the highest beta to try
           detm: the lattice detmn
           norm: the target vector norm
           dim: the lattice dim
    The function applies Binary search over the values starting from the
    lowest value to the highest one and return the value that satsifies the condition
    
    This function is used in finding the value of beta in the 2016 estimator
    """
    #print("low beta: ", low_beta)
    #print("high beta: ", high_beta)
    ls = left_side(low_beta,dim,norm)
    rs = right_side(low_beta,dim,detmn)
    #if loweset beta satisfies return the value
    if(low_beta>=(high_beta-1)):
        if ls<rs:
            return low_beta
        else:
            return high_beta
    
    mid = np.ceil((low_beta+high_beta)/2)
    ls = left_side(mid,dim,norm)
    rs = right_side(mid,dim,detmn)
    if ls<rs:
        t = Binary_search(left_side, right_side, low_beta,mid, detmn, norm,dim)
    else:
        t = Binary_search(left_side, right_side, mid, high_beta, detmn, norm,dim)
    return t
    

In [25]:
def get_beta_2016_estimator(N, d, q, p=3,cyclic=1):
    """
    Input: - N: the order of the group in the case of the cyclic group and half the order in the
            case of the dihedral group
           - d: that defines the secrect key space T(d,d)[d is half d in the case of the dihedral group]
           - q: the modulus.
           - p: often =3
           - cyclic: refers if the cyclic group is used 
    Output: the value of beta that gives the correct estimation in the 2016 estimation(binary searched)
    """
    dim = N*2
    detmn = get_qary_det(q,N)
    dg = int(N/3) ##dg is one third the order of the group
    norm = get_private_key_norm(d, dg, cyclic) ##norm of the cyclic/dihedral group for the corresponding n
    #print("dim: {}, q: {}, norm: {}".format(dim,q,norm))
    #print("detmn: {}".format(detmn))
    return Binary_search(left_side, right_side, 100,dim, detmn, norm,dim) ##locate beta through binary search
   

In [26]:
def comb(n,k):
    """
    Input: n, k where n>k
    Output: ncr(n,k)
    """
    if n<k:
        return 0
    
    t1 = factorial(n)
    t2 = factorial(k)*factorial(n-k)
    
    
    return int(t1/t2)

## All the value of n that can serve to build the cryptosystem
## These values of n increase the probability of having units

In [27]:
N1 = [ 101, 107, 131, 139, 149, 163, 173, 179, 181, 197,
 211, 227, 269, 293, 317, 347, 349, 373, 379, 389,
419, 421, 443, 461, 467, 491, 509, 523, 541, 547,
557, 563, 587, 613, 619, 653, 659, 661, 677, 701,
709, 757, 773, 787, 797, 821, 827, 829, 853, 859,
877, 883, 907, 941, 947, 1019, 1061, 1091, 1109, 1117,
1123, 1171, 1187, 1213, 1229, 1237, 1259, 1277, 1283, 1291,
1301, 1307, 1373, 1381, 1427, 1451, 1453, 1483, 1493, 1499,
1523, 1531, 1549, 1571, 1619, 1621, 1637, 1667, 1669, 1693,
1733, 1741, 1747, 1787, 1861, 1867, 1877, 1901, 1907, 1931]
     
     
     
N2 = [103, 137, 167, 191, 193, 199, 239, 263, 271, 311,
 313, 359, 367, 383, 401, 409, 449, 463, 479, 487,
503, 521, 569, 599, 607, 647, 719, 743, 751, 761,
769, 809, 823, 839, 857, 863, 887, 929, 967, 977,
983, 991, 1009, 1031, 1039, 1063, 1087, 1129, 1151, 1223,
1231, 1279, 1297, 1303, 1319, 1361, 1367, 1409, 1439, 1447,
1487, 1489, 1511, 1543, 1559, 1567, 1583, 1607, 1663, 1697,
1759, 1783, 1823, 1847, 1871, 1873, 1879, 1951, 1993, 2039]     


Ni = N1+N2     

In [28]:
Ni.sort()

In [29]:
def get2_order(N):
    """
    return the order of 2 in Z^*_{N}
    """
   
    for j in range(1,N):
        if 2^j%N==1:
            return j
    return -1
        
    

In [30]:
def get_all_N_order2(Ni):
    """
    Input:  - The list of all the prime numbers
    Output: - The list of the prime where 2 has the order n-1
    """
    s = set({})
    for n in Ni:
        if get2_order(n)==n-1:
            s.add(n)
    return s
            

In [31]:
order2 =get_all_N_order2(Ni)

In [32]:
t = []
for i in order2:
    t.append(i)
t.sort()
print(t)

[101, 107, 131, 139, 149, 163, 173, 179, 181, 197, 211, 227, 269, 293, 317, 347, 349, 373, 379, 389, 419, 421, 443, 461, 467, 491, 509, 523, 541, 547, 557, 563, 587, 613, 619, 653, 659, 661, 677, 701, 709, 757, 773, 787, 797, 821, 827, 829, 853, 859, 877, 883, 907, 941, 947, 1019, 1061, 1091, 1109, 1117, 1123, 1171, 1187, 1213, 1229, 1237, 1259, 1277, 1283, 1291, 1301, 1307, 1373, 1381, 1427, 1451, 1453, 1483, 1493, 1499, 1523, 1531, 1549, 1571, 1619, 1621, 1637, 1667, 1669, 1693, 1733, 1741, 1747, 1787, 1861, 1867, 1877, 1901, 1907, 1931]


In [33]:
def get_combinatorial_search_actual_dihedral(N,d):
    """
    Input - N: half the order of the dihedral group
          - d: defines d in T(d,d) 
    
    Output: - the combinatorial search cost for the private key assuming one layer of reduction is possible
    """
    #d = 2*d #[d is the orginal d that defines the orginal space, passed d is half of it]
    d3 = int(d/3)
    d6 = int(d/6)
    return (comb(N,d3)*comb(N-d3,d3)*comb(N-2*d3, d6)*comb(N-5*d6, d6))

In [34]:
def get_combinatorial_search_actual_dihedral(N,d):
    """
    Input - N: half the order of the dihedral group
          - d: defines d in T(d,d) (df)
    
    Output: - the combinatorial search cost for the private key assuming one layer of reduction is possible
    """
    #d = 2*d #[d is the orginal d that defines the orginal space, passed d is half of it]
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    return (comb(N,d1)*comb(N-d1,d1)*comb(N-2*d1, d3)*comb(N-2*d1-d3, d3))

In [35]:
def get_combinatorial_search_actual_dihedral_log(N,d):
    """
    Input - N: half the order of the dihedral group
          - d: defines d in T(d,d) 
    
    Output: - the combinatorial search cost for the private key assuming one layer of reduction is possible
    """
    #d = 2*d #[d is the orginal d that defines the orginal space, passed d is half of it]
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    log1 = log_2(comb(N,d1))
    log2 = log_2(comb(N-d1,d1))
    log3 = log_2(comb(N-2*d1, d3))
    log4 = log_2(comb(N-2*d1-d3, d3))
    return (log1+log2+log3+log4)

In [36]:
def get_combinatorial_search(N,d, cyclic=1, classical=1, max_depth=96):
    """
    Input:  - N: the order of the group in the case of the cyclic group and half the order in the
            case of the dihedral group
            - d: the total number of 1/-1/0
            - cyclic: refers if we are using the cyclic group as the underlying group
            - classical: indicates if the cost is calculated according to classical or quantum estimation(defaul=1 classical calcultion)
    Output: the combinatoral search cost for the private key
    In our case: we assum that the key is sample form T(d,d) to match the calculations 
    of the first round of NTRU submission
    (n d)  (n-d   d) 
    """
    classical_factor = 0.5
    quantum_factor = 0.25
    if cyclic==1:
        combo = (comb(N,d)*comb(N-d,d))
        
        n = N
    else:
        ## the calculation inside the get_combinatorial_function happens with respect to the orginal d)
        combo = get_combinatorial_search_actual_dihedral(N,2*d)
        n =2*N
    
    if classical==1 or (combo>2^max_depth and max_depth!=-1): ##-1 means no constraint
        factor = classical_factor ##first meet-in-the-middle estimation, replace it later with 0.3
    else:
        print("quantum factor")
        factor = quantum_factor ##grover algorithm, replace it later with 0.19
        
    pow_log2 = log_2(combo/n)
    
    return int(factor*pow_log2)   ###return the power 

In [37]:
def get_combinatorial_search_log2(N,d, cyclic=1, classical=1, max_depth=96):
    """
    Input:  - N: the order of the group in the case of the cyclic group and half the order in the
            case of the dihedral group
            - d: the total number of 1/-1/0
            - cyclic: refers if we are using the cyclic group as the underlying group
            - classical: indicates if the cost is calculated according to classical or quantum estimation(defaul=1 classical calcultion)
    Output: the combinatoral search cost for the private key
    In our case: we assum that the key is sample form T(d,d) to match the calculations 
    of the first round of NTRU submission
    (n d)  (n-d   d) 
    """
    classical_factor = 0.5
    quantum_factor = 0.25
    if cyclic==1:
        log1 = log_2(comb(N,d))
        log2 = log_2(comb(N-d,d))
        combo = log1+log2
        
        n = log_2(N)
    else:
        ## the calculation inside the get_combinatorial_function happens with respect to the orginal d)
        combo = get_combinatorial_search_actual_dihedral_log(N,2*d)
        n = log_2(2*N)
    
    if classical==1 or (combo>2^max_depth and max_depth!=-1): ##-1 means no constraint
        factor = classical_factor ##first meet-in-the-middle estimation, replace it later with 0.3
    else:
        print("quantum factor")
        factor = quantum_factor ##grover algorithm, replace it later with 0.19
        
    pow_log2 = combo-n
    
    return int(factor*pow_log2)   ###return the power 

## Cyclic group estimation

## Define some functions to be used by hybird attack

In [38]:
def probability(N,d,k,a,b):
    """
    Input: - N  the number of coefficients.
           - K: a parameter used in the hybird search.
           - d: the parameter that defines the space form which we are taking the key T(d,d)
           - a,b: a: refers to the number of 1s and b refers to the number of -1.
    
    The function calculates the probability of havig the last K elements in a ternary sequence
    having a ones and b minus ones exactly, this probabity takes into consideration calculationg the 
    probabity for one representative of v_{a,b}, that is why we are dividing by (K a) (K-a b)
    
    The probability is calculated as:
    
    p(v_{a,b}) = |S_{pi}(a,b)|/(|S| nCr(K,a)*nCr(K-a,b)) = {nCr(N-k, d-a)*nCr(N-K-d+a, d-b)}/{nCr(N,d)*nCr(N-d,d)}
    """
    
    t1 = comb(N-k, d-a)*comb(N-k-d+a, d-b)
    t2 = comb(N,d)*comb(N-d, d)
    
    return (float(t1/t2))

### Probability for Dihedral

In [39]:
def probability_for_dihedral(N,d,k,a1,a2,a3,a4):
    """
    Input:  - N(half the order of the dihedral group).
            - K: a parameter used in the hybird search
            - d: the parameter that defines the space form which we are taking the key T(d,d): the orginal d [it is passed as the orginal value]
            - a,b: a: refers to the number of 1s and b refers to the number of -1.
            - c,d: c: refers to the number of 2s and d refers to the number of -2.
            The function calculates the probability of havig the last K elements in a ternary sequence
            having a ones and b minus ones exactly, this probabity takes into consideration calculationg the 
            probabity for one representative of v_{a,b}, that is why we are dividing by (K a) (K-a b)
    
            - The probability is calculated as in the paper
    
    """
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    
    t1 = comb(N-k, d1-a1)*comb(N-k-d1+a1, d1-a2)*comb(N-k-2*d1+a1+a2, d3-a3)*comb(N-k-2*d1-d3+a1+a2+a3,d3-a4)
    t2 = get_combinatorial_search_actual_dihedral(N,d) ###already passed as the orginal value no need to double
    #print("inside probability: t2={}, N={}, d={}".format(t2,N,d))
    return (float(t1/t2))

In [40]:
def probability_for_dihedral_log2(N,d,k,a1,a2,a3,a4):
    """
    Input:  - N(half the order of the dihedral group).
            - K: a parameter used in the hybird search
            - d: the parameter that defines the space form which we are taking the key T(d,d): the orginal d [it is passed as the orginal value]
            - a,b: a: refers to the number of 1s and b refers to the number of -1.
            - c,d: c: refers to the number of 2s and d refers to the number of -2.
            The function calculates the probability of havig the last K elements in a ternary sequence
            having a ones and b minus ones exactly, this probabity takes into consideration calculationg the 
            probabity for one representative of v_{a,b}, that is why we are dividing by (K a) (K-a b)
    
            - The probability is calculated as in the paper and the returned value is the power of two
    
    """
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    
    log1 = log_2(comb(N-k, d1-a1))
    log2 = log_2(comb(N-k-d1+a1, d1-a2))
    log3 = log_2(comb(N-k-2*d1+a1+a2, d3-a3))
    log4 = log_2(comb(N-k-2*d1-d3+a1+a2+a3,d3-a4))
    t1 = log1+log2+log3+log4
    t2 = get_combinatorial_search_actual_dihedral_log(N,d) ###already passed as the orginal value no need to double
    #print("inside probability: t2={}, N={}, d={}".format(t2,N,d))
    return (t1-t2)

In [41]:
def log_2(prob):
    #print("inside log_2: ",prob)
    if prob==0:
        #print("zero")
        return 0
    else:
        return math.log2(prob)

In [42]:
def get_Hp_cyclic(N,d,k):
    """
    Input: - N the number of coefficients.
           - K a parameter used in the hybird search.
           - d the parameter that defines the space form which we are taking the key T(d,d)
      
    Output: - estimation of the search space Hp
    """
    t =0
    #a,b: a: refers to the number of 1s and b refers to the number of -1.
    for a in range(0,d+1):
        for b in range(0,d+1):
            #print("a: {}, b: {}, k: {}".format(a,b,k))
            #print(comb(k,a)*comb(k-a,b))
            #print(probability(N,d,k,a,b))
            #print(log_2(probability(N,d,k,a,b)))
            t= comb(k,a)*comb(k-a,b)*probability(N,d,k,a,b)*log_2(probability(N,d,k,a,b))+t
            #print("t: ", t)
    return -1*t

In [43]:
def get_Hp_cyclic_log(N,d,k):
    """
    Input: - N the number of coefficients.
           - K a parameter used in the hybird search.
           - d the parameter that defines the space form which we are taking the key T(d,d)
      
    Output: - estimation of the search space Hp
    """
    t =0
    #a,b: a: refers to the number of 1s and b refers to the number of -1.
    for a in range(0,d+1):
        for b in range(0,d+1):
            #print("a: {}, b: {}, k: {}".format(a,b,k))
            #print(comb(k,a)*comb(k-a,b))
            #print(probability(N,d,k,a,b))
            #print(log_2(probability(N,d,k,a,b)))
            pr = probability(N,d,k,a,b)
            log1 = log_2(comb(k,a))
            #print("comb: ", comb(k-a,b))
            log2 = log_2(comb(k-a,b))
            log3 = log_2(pr)
            first_part = 2^(log1+log2+log3)
            t= first_part*log3+t
            #print("t: ", t)
    return -1*t

In [44]:
def get_Hp_dihedral(N,dprime,k):
    """
    Input:  - N (half the order of the dihedral group).
            - K: a parameter used in the hybird search.
            - dprime: half of d that defines f in T(d,d)
            
    Output: - estimation of the search space Hp
    """
    
    print("inside the get_hp function!")
    d =dprime*2
    t =0
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    for a1 in range(0,d1+1):
        for a2 in range(0,d1+1):
            for a3 in range(0,d3+1):
                for a4 in range(0,d3+1):
                    prob = probability_for_dihedral(N,d,k,a1,a2,a3,a4)
                    cons = comb(k,a1)*comb(k-a1,a2)*comb(k-a1-a2,a3)*comb(k-a1-a2-a3,a4)
                    t= cons*prob*log_2(prob)+t
            #print("t: ", t)
    return -1*t

In [45]:
def get_Hp_dihedral_log(N,dprime,k):
    """
    Input:  - N (half the order of the dihedral group).
            - K: a parameter used in the hybird search.
            - dprime: half of d that defines f in T(d,d)
            
    Output: - estimation of the search space Hp
    """
    
    print("inside the get_hp function!")
    d =dprime*2
    t =0
    d1 = int((d*(N-d))/N)
    d3 = int(d^2/(4*N))
    for a1 in range(0,d1+1):
        for a2 in range(0,d1+1):
            for a3 in range(0,d3+1):
                for a4 in range(0,d3+1):
                    prob = probability_for_dihedral_log2(N,d,k,a1,a2,a3,a4)  ##log2(prop)
                    #print("prob: ", 2^prob)
                    #print("the orginal: ", probability_for_dihedral(N,d,k,a1,a2,a3,a4))
                    log1 = log_2(comb(k,a1))
                    log2 = log_2(comb(k-a1,a2))
                    log3 = log_2(comb(k-a1-a2,a3))
                    log4 = log_2(comb(k-a1-a2-a3,a4))
                    #cons = log1+log2+log3+log4
                    first_part = 2^(log1+log2+log3+log4+prob)
                    #t1 = prob+cons ### power of two
                    t= first_part*prob+t
            #print("t: ", t)
    return -1*t

In [46]:
def get_Hp(N,d,k, cyclic=1):
    """
    Input: N : the number of coefficients.
    K: a parameter used in the hybird search.
    d: the parameter that defines the space form which we are taking the key T(d,d)
    cyclic: refers if we are using the cyclic group as underlying group
    """
    if cyclic==1:
        Hp = get_Hp_cyclic(N,d,k)
    else:
        Hp = get_Hp_dihedral(N,d,k) ##inside the function, the value of d is doubled.
        
    return Hp

In [47]:
def get_Hp_log(N,d,k, cyclic=1):
    """
    Input: N : the number of coefficients.
    K: a parameter used in the hybird search.
    d: the parameter that defines the space form which we are taking the key T(d,d)
    cyclic: refers if we are using the cyclic group as underlying group
    """
    if cyclic==1:
        Hp = get_Hp_cyclic_log(N,d,k)
    else:
        Hp = get_Hp_dihedral_log(N,d,k) ##inside the function, the value of d is doubled.
        
    return Hp

In [48]:
def search_with_circuit_depth(Hp, n, quantum_factor,classical_factor, maxdepth, classical):
    """
    Input:  - Hp: the estimation of the search space
            - n : the order of the group
            - factor: the factor of the quantum search speed up
            - maxdepth: the maximum depth of the quantum circuit
            if the maximum depth is -1, there is no constraint on the search cost
            - classical: indicates the search either classical or quantum
    Output: - the search cost with the restriction of the quantum depth 
    """
    if classical or (Hp>maxdepth and maxdepth !=-1):
        t = classical_factor*(Hp-math.log2(n)) 
    else:
        t = quantum_factor*(Hp-math.log2(n)) 
    
    return t

In [49]:
def get_search_cost(N,d,k, cyclic=1, classical=1):
    """
    Input: - N : the number of coefficients.
           - K: a parameter used in the hybird search.
           - d: the parameter that defines the space form which we are taking the key T(d,d).
           - cyclic: refers if we are using the cyclic or the dihedral group defualt=1 for cyclic group
           - classical = 1 refers that the estimation is calculated with respect to the classical factor otherwise quantum factor
    
    Output: The final cost of search for a specific value K.
    """
    print("inside get search cost, classical: ", classical)
    classical_factor = 0.5
    quantum_factor = 0.25
    if cyclic==1:
        Hp = get_Hp(N,d,k)
        n = N
    else:
        Hp = get_Hp_dihedral(N,d,k) ## get the estimation on the search space for the case of the dihedral
        n= 2*N
    
#     if classical==1:
#         factor = classical_factor ##Normal MITM, replace it later by 0.3 to refer to May's attack
#         t 
#     else:
#         print("quantum factor in HP")
#         factor = quantum_factor ##replace it later by the 0.19 to refer to the quantum version of May's attack
#     t = factor*(Hp-math.log2(n)) 

    t = search_with_circuit_depth(Hp, n, quantum_factor,classical_factor, maxdepth=96, classical=classical)
    
    return round(t)

In [50]:
def get_search_cost_log(N,d,k, cyclic=1, classical=1):
    """
    Input: - N : the number of coefficients.
           - K: a parameter used in the hybird search.
           - d: the parameter that defines the space form which we are taking the key T(d,d).
           - cyclic: refers if we are using the cyclic or the dihedral group defualt=1 for cyclic group
           - classical = 1 refers that the estimation is calculated with respect to the classical factor otherwise quantum factor
    
    Output: The final cost of search for a specific value K.
    """
    print("inside get search cost, classical: ", classical)
    classical_factor = 0.5
    quantum_factor = 0.25
    if cyclic==1:
        Hp = get_Hp_log(N,d,k)
        n = N
    else:
        Hp = get_Hp_dihedral_log(N,d,k) ## get the estimation on the search space for the case of the dihedral
        n= 2*N
    
#     if classical==1:
#         factor = classical_factor ##Normal MITM, replace it later by 0.3 to refer to May's attack
#         t 
#     else:
#         print("quantum factor in HP")
#         factor = quantum_factor ##replace it later by the 0.19 to refer to the quantum version of May's attack
#     t = factor*(Hp-math.log2(n)) 

    t = search_with_circuit_depth(Hp, n, quantum_factor,classical_factor, maxdepth=96, classical=classical)
    
    return round(t)

In [51]:
def get_search_cost_log(N,d,k, cyclic=1, classical=1):
    """
    Input: - N : the number of coefficients.
           - K: a parameter used in the hybird search.
           - d: the parameter that defines the space form which we are taking the key T(d,d).
           - cyclic: refers if we are using the cyclic or the dihedral group defualt=1 for cyclic group
           - classical = 1 refers that the estimation is calculated with respect to the classical factor otherwise quantum factor
    
    Output: The final cost of search for a specific value K.
    """
    print("inside get search cost, classical: ", classical)
    classical_factor = 0.5
    quantum_factor = 0.25
    if cyclic==1:
        Hp = get_Hp_log(N,d,k)
        n = N
    else:
        Hp = get_Hp_dihedral_log(N,d,k) ## get the estimation on the search space for the case of the dihedral
        n= 2*N
    
#     if classical==1:
#         factor = classical_factor ##Normal MITM, replace it later by 0.3 to refer to May's attack
#         t 
#     else:
#         print("quantum factor in HP")
#         factor = quantum_factor ##replace it later by the 0.19 to refer to the quantum version of May's attack
#     t = factor*(Hp-math.log2(n)) 

    t = search_with_circuit_depth(Hp, n, quantum_factor,classical_factor, maxdepth=96, classical=classical)
    
    return round(t)

In [52]:
def get_delta(N,q,r1,k, cyclic=1):
    """
    Input: 
        - N: the number of coefficients(for dihedral: half the order of the dihedral group).
        - q: the used modulo.
        - r1: the starting column to start the reduction.
        - k: the parameter that specifies the number of the coefficients to search for.
        - cyclic: refers we are using the cyclic group otherwise dihedral
    Output:
        - nu: the upper bound on the root herimite facator needed so that the hybird attack works successfully.
        
    The returned nu is the minmum required delta to make the reduction work, any smaller value for this nu 
    will make the attack successful.
    
    
    """
    print("inside the get_delta function: N={}, q={}, r={}, k={}, cyclic={} ".format(N,q,r1,k,cyclic))
    r2 = 2*N-k
    s = N-(r1+r2)/2
    N1 = (r2-r1)/2
    mult=1
    if cyclic==1: 
        mult=2##if we are using cyclic then the value of t will be larger, for dihedral it is 
               ### smaller which means the attack in the case of the dihedral is more difficult to perfrom i.e., the 
               ## reduction algorithm needs to produce smaller value of delta to make the hybrid attack feasible
       
    t = ((N1+s)/(4*N1^2))*math.log2(q)-1/(mult*N1)
    
    return 2^float(t)

In [53]:
def get_beta_from_delta_bs(delta, low_beta=30, high_beta=2000):
    
    """
    Input: delta, the minimum required delta for the attack to work
    Output: the corresponding beta that acheive such delta,
    we search for such beta using the binary search
    """
    
    #print("low beta: ", low_beta)
    #print("high beta: ", high_beta)
    
    #if loweset beta satisfies return the value
    if(low_beta>=(high_beta-1)):
        ls = (low_beta/(2*np.pi*np.e) *(np.pi*low_beta)^(1/low_beta))^(1/(2*(low_beta-1)))
        rs = delta
        if ls<rs:
            return low_beta
        else:
            ls = (high_beta/(2*np.pi*np.e) *(np.pi*high_beta)^(1/high_beta))^(1/(2*(high_beta-1)))
            rs = delta
            if ls<rs:
                return high_beta
            return "failure" ##failure means we couldn't find a beta in the domain that can acheives the equ
    
    mid_beta = np.ceil((low_beta+high_beta)/2)
    ls = (mid_beta/(2*np.pi*np.e) *(np.pi*mid_beta)^(1/mid_beta))^(1/(2*(mid_beta-1)))
    rs = delta
    if ls<rs:
        t = get_beta_from_delta_bs(delta, low_beta, mid_beta)
    else:
        t = get_beta_from_delta_bs(delta, mid_beta, high_beta)
    return t
    
    
    
    

In [54]:
def all_classical_enumeration(beta, options=[0,1], d=None):
    """
    Input: beta, the blocksize needed to retreive the key.
           d: the dimension of the lattice.
           options: options that specifies which model to use
    Output: the security estimation of the model according to all the classical 
    enumeration we know
    """
    classical_enum = []
    #print("options: ", options)
    if options.count(0):
        classical_enum1 = round(0.187*beta*math.log2(beta)-1.019*beta+16.1)
        classical_enum.append(classical_enum1)
    if options.count(1):
        classical_enum2 = round(0.000784*beta^2+0.366*beta-0.9+math.log2(8*d))
        classical_enum.append(classical_enum2)
        
    return classical_enum
    
    
    
    

In [55]:
def all_classical_sieving(beta, options=[0,1,2,3,4], d=None):
    """
    Input: beta, the blocksize needed to retreive the key.
           d: the dimension of the lattice.
           options: options that specifies which model to use
    Output: the security estimation of the model according to all the classical 
    seiving we know
    """
    classical_sieving = []
    #print("inside classical seiving: ", options)
    if options.count(0):
        classical_sieving1 = round(0.292*beta)
        classical_sieving.append(classical_sieving1)
        
    if options.count(1):
        classical_sieving2 = round(0.292*beta +math.log2(beta))
        classical_sieving.append(classical_sieving2)
        
    if options.count(2):
        classical_sieving3 = round(0.292*beta + 16.4)
        classical_sieving.append(classical_sieving3)
        
    if options.count(3):
        classical_sieving4 = round(0.292*beta + 16.4 + math.log2(8*d))
        classical_sieving.append(classical_sieving4)
        
    if options.count(4):
        classical_sieving5 = round(0.368*beta)
        classical_sieving.append(classical_sieving5)
        
    return classical_sieving
    
    

In [56]:
def all_quantum_enumeration(beta, options=[0,1], d=None):
    
    """
     Input: beta, the blocksize needed to retreive the key.
           d: the dimension of the lattice.
           options: options that specifies which model to use
    Output: the security estimation of the model according to all the quantum 
    enumeration we know
    """
    quantum_enum = []
    if options.count(0):
        quantum_enum1 = round(0.5*( 0.187*beta*math.log2(beta)- 1.019*beta + 16.1 ))
        quantum_enum.append(quantum_enum1)
    if options.count(1):
        quantum_enum2 = round(0.125*beta*math.log2(beta)- 0.755*beta + 2.25)
        quantum_enum.append(quantum_enum2)
        
    return quantum_enum

In [57]:
def all_quantum_seiving(beta, options=[0,1,2,3,4], d=None):
    
    """
     Input: beta, the blocksize needed to retreive the key.
           d: the dimension of the lattice.
           options: options that specifies which model to use
    Output: the security estimation of the model according to all the quantum 
    sieving we know
    """
    quantum_sieving = []
    if options.count(0):
        quantum_sieving1 = round(0.265*beta) 
        quantum_sieving.append(quantum_sieving1)
        
    if options.count(1):
        quantum_sieving2 = round(0.265*beta +math.log2(beta))
        quantum_sieving.append(quantum_sieving2)
        
    if options.count(2):
        qunatum_sieving3 = round(0.265*beta+16.4)
        quantum_sieving.append(qunatum_sieving3)
        
    if options.count(3):
        quantum_sieving4 = round(0.265*beta+16.4+math.log2(8*d))
        quantum_sieving.append(quantum_sieving4)
    
    if options.count(4):
        quantum_sieving5 = round(0.2975*beta)
        quantum_sieving.append(quantum_sieving5)
        
    return quantum_sieving
        
        
    return quantum_enum

In [58]:
def model_formula(_id):
    """
    Input: the id of the model:
    Output: the formula of the model.
    """
    
    switcher = {
        "ce1": "0.187β*log2(β)-1.019*β+16.1",
        "ce2": "0.000784β^2+0.366β-0.9+log2(8d)",
        
        "cs1": "0.292β",
        "cs2": "0.292β +log2(β)",
        "cs3": "0.292β + 16.4" ,
        "cs4": "0.292β + 16.4 + log2(8d)",
        "cs5": "0.368β",
        
        "qe1": "1/2( 0.187β*log2(β)- 1.019β + 16.1 )",
        "qe2": "0.125β*log2(β)- 0.755β + 2.25",
        
        "qs1": "0.265β ",
        "qs2": "0.265β +log2(β)",
        "qs3": "0.265β+16.4",
        "qs4": "0.265β+16.4+log2(8d)",
        "qs5":" 0.2975β"
        
    }
    
    return switcher.get(_id, "nothing")

In [59]:
def all_models(beta, d):
    """
    Input: beta the blocksize
           d: the lattice dimension
    The function evaluates the security according to all the models and print
    the estimation
    """
    
    print("classical enumeration: ")
    options = [0,1]
    resu =  all_classical_enumeration(beta, options , d)
    for i in options:
        print("Model {} 's estimation is: {} ".format(model_formula("ce"+str(i+1)), resu[i]))
        
    print(" ----------------------- ")
    
    print("classical sieving: ")
    options = [0,1,2,3,4]
    resu = all_classical_sieving(beta, options, d)
    for i in options:
              print("Model {} 's estimation is: {} ".format(model_formula("cs"+str(i+1)), resu[i]))
        
    print("------------------------ ")
    
    print("quantum enumeration: ")
    
    options = [0,1]
    resu = all_quantum_enumeration(beta, options, d)
    for i in options:
        print("Model {} 's estimation is: {} ".format(model_formula("qe"+str(i+1)), resu[i]))
        
    print("------------------------ ")
    
    print("quantum sieving:")
    options = [0,1,2,3,4]
    resu = all_quantum_seiving(beta, options, d)
    for i in options:
          print("Model {} 's estimation is: {} ".format(model_formula("qs"+str(i+1)), resu[i]))
        
        
    
    

In [60]:
def best_classical_enumeration(beta):
    """
    Input: beta the blocksize needed to do the reduction
    Output: the estimation according to some classical enumeration model
    According to NTRUEncrypt first round, they are considering this model to give the best classical security estimation
    """
    #equ = 0.000784314*beta^2+0.366078*beta-6.125
    equ  = round(0.187*beta*math.log2(beta)-1.019*beta+16.1)
    return equ

In [61]:
def best_quantum_enumeration(beta):
    """
    Input: beta the blocksize needed to do the reduction
    Output: the estimation according to some quantum enumeration model
    According to NTRUEncrypt first round, they are considering this model to give the best quantum security estimation
    """
    #equ = 0.000784314*beta^2+0.366078*beta-6.125
    equ  = round(0.5*(0.187*beta*math.log2(beta)-1.019*beta+16.1))
    return equ

In [62]:
def best_classical_seiving(beta):
    """
    Input: beta: the blocksize
    Output: the estimation according to some classical seving model
    Please note that according to the discusion of NTRUEncrypt first round: 
    For NTRUEncrypt first round, the classical seving is not pratical and not used since the memory requirement are very high
    """
    
    equ = round(0.292*beta)
    
    return equ

In [63]:
def best_quantum_seiving(beta):
    
    """
    Input: beta, the blocksize
    Output: the estimation according to the best quantum seiving model
    """
    equ = round(0.265*beta)
    
    return equ

In [64]:
def plausible_quantum_seiving(beta):
    
    """
    Input: beta the blocksize
    Output: the quantum seivign estimation according to the plausible quantum model.
    Pleae note that this model is used to estimate the parameters of NTRU first round.
    """
    equ = round(0.2075*beta)
    
    return equ

In [65]:
def reduction_model(beta, opt):
    """
    Input:- beta the blocksize which is the input of the reduction model
          - opt: the option for reduction
              1: for classical sieving 
              2: for quanutm sieving
              3: for classical enumeration
              4: for quantum enumeration  
    """
    #print("The option is ", opt)
    if (opt==1):
        return best_classical_seiving(beta)
    elif(opt==2):
        return best_quantum_seiving(beta)
    elif(opt==3):
        return best_classical_enumeration(beta)
    elif(opt==4):
        return best_quantum_enumeration(beta)
        

In [66]:
def get_model_description(opt):
    """
    Input: the option according to it the reduction happend
    Output: a text description
    
    """
    if (opt==1):
        return "classical_seiving"
    elif(opt==2):
        return "quantum_seiving"
    elif(opt==3):
        return "classical_enumeration"
    elif(opt==4):
        return "quantum_enumeration"

In [82]:
def get_group_description(cyclic, twisted=False):
    """
    Input: cyclic =1 or 0
    Output: a text description
    
    """
    if (cyclic==1):
        if twisted==False:
            return "cyclic"
        else:
            return "twisted_dihedral"
    else:
        return "dihedral"

In [68]:
def get_threshold_cost(lamda):
    """
    Input: - lamda, the level of security.
    Output: the core-svp threshold that means the level of security
    
    For now, we keep the threshold as the lamda
    """
    
    return lamda

In [69]:
def Binary_search_for_hybrid(N,q,d,lamda,old_minimum, low_k, high_k,left=1,fast_quick=1, reduction_opt=1, r1=-1, cyclic=1):
    
    """
    This function is supposed to estimate the cost of the hybird attack according to some enumeration/seiving model
    The search is conducted using Binary search for faster results
    
    Input: 
        - old minimum: the minimum cost of the evaluation till now; the initial value is 10000 
        - N: the number of coefficients in the case of the cyclic group(half the order of the dihedral group since we are doing the search in the smaller lattices)
        - q: the modulo
        - d: the parameter that defines the space frow which the key is sampled[half of d for dihedral] 
        - lamda the required level of security to acheive
        - fast_quick: 1 means when the cost at any point becomes smaller than lamda exit otherwise 
                      continue processing until you get the exact value of the attack
        - low_k: the smallest value of k
        - high_k:the highest possible value of k it should be initialized as N 
        - reduction_opt: refers to the reduction model to apply: default 1 refers to the classical sieving
        - cyclic: refers if the underlying group is cyclic 
        
    Output: the hyrid attack estimation
        
    Trying to figure out the best value of k that makes the hybird attacks successful
    
    """
    #print("r1: ", r1)
    #print("low k: ", low_k)
    #print("high_k: ", high_k)
    classical = is_it_classical(reduction_opt)
    print("inside binary search: classiacal is: ", classical)
    if r1==-1: #not initialized r1
        r1 = min(lamda, int(N/2))
    if low_k>=(high_k-1):
        search_cost = get_search_cost_log(N,d,low_k, cyclic, classical)
        print("search cost: ", search_cost)
        delta_for_reduction = get_delta(N,q,r1,low_k,cyclic)
        print("delat for reduction: ", delta_for_reduction)
        beta = get_beta_from_delta_bs(delta_for_reduction)
        print("beta: ", beta)
        reduction_cost = reduction_model(beta, reduction_opt)   ##classical sieving
        print("reduction cost: ", reduction_cost)
        maxi = max(search_cost, reduction_cost)

        if low_k==high_k:
            return maxi, low_k, r1, beta
        else: ## There is a differnce one between low_k and high_k
            search_cost = get_search_cost_log(N,d,high_k,cyclic,classical)
            print("search cost: ", search_cost)
            delta_for_reduction = get_delta(N,q,r1,high_k,cyclic)
            print("delta for reduction: ", delta_for_reduction)
            beta1 = get_beta_from_delta_bs(delta_for_reduction)
            print("beta: ", beta)
            reduction_cost = reduction_model(beta1, reduction_opt)   ##classical sieving
            print("reduction cost: ", reduction_cost)
            maxi2 = max(search_cost, reduction_cost)
            if maxi2<maxi:
                return maxi2, high_k, r1, beta1
            else:
                return maxi, low_k, r1, beta


    mid_k = int(np.ceil((low_k+high_k)/2))
    print("N = {}, d = {}, mid_k = {}".format(N,d,mid_k))
    search_cost = get_search_cost_log(N,d,mid_k,cyclic, classical)
    print("search cost: ", search_cost)
    delta_for_reduction = get_delta(N,q,r1,mid_k,cyclic)
    print("delat for reduction: ", delta_for_reduction)
    beta = get_beta_from_delta_bs(delta_for_reduction)
    print("beta: ", beta)
    reduction_cost = reduction_model(beta, reduction_opt)   ##seiving is the used model classically and quantumly
    print("reduction cost: ", reduction_cost)
    Min_t = max(search_cost, reduction_cost) ##can't acheive the required level of security
    if Min_t<get_threshold_cost(lamda) and fast_quick:
        return Min_t, mid_k, r1, beta
    if Min_t<old_minimum:
        if Min_t==search_cost:
            high_k = mid_k
            left = 1 #go left smaller values of k
        else:
            low_k = mid_k
            left =0 #go right larger values of k
    else: #Min_t is not smaller than the old minimum

        if left==1: ## We are coming from left 
            low_k = mid_k
            #left = 0 ## We are goin towards right, larger values of k
        else:
            high_k = mid_k
            #left = 1 ## We are going towards left, smaller values of k


    return Binary_search_for_hybrid(N,q,d,lamda,Min_t, low_k, high_k,left,fast_quick, reduction_opt,r1,cyclic)     


In [70]:
def final_security_estimation(N,q,d,lamda,beta, reduction_option=1, fast_quick=1, hybird=True,cyclic=1):
    """
    - The function calls the best models to estimate the classical security, quantum security to solve the uSVP problem
              and the best cost to solve the hybird attack.
    Input: 
            - beta: the blocksize.
            - hybird: to indicates if we are using the hybird estimation or not default=False i.e, we consider only primal attack.
            - reduction_opt: refers to the reduction model to apply: default 1 refers to the classical sieving
            - fast_quick: 1 means when the cost at any point becomes smaller than lamda exit otherwise 
                      continue processing until you get the exact value of the attack
            - cyclic: refers if the underlying group is cyclic or dihedral
                 
    Output: - arr contains the estimation according the required options in the input.        
    """
    
    aux = []
    arr = []
    reduction_cost = reduction_model(beta, reduction_option)
    arr+=[reduction_cost]
    if  reduction_cost< get_threshold_cost(lamda) and fast_quick:
        return arr, aux
    if hybird:
            best_ch,saved_k, saved_r1, beta = Binary_search_for_hybrid(N,q,d,lamda,1000000, 10, N,left=1,fast_quick=fast_quick, reduction_opt=reduction_option, r1=-1,cyclic=cyclic)
            #best_classical_hybird(N,q,d,lamda, fast_quick)
            arr=[reduction_cost, best_ch]
            aux+=[saved_k, saved_r1, beta]    
# 
    return arr,aux

In [71]:
def is_it_classical(reduction_option):
    """
    Input: - reduction_option: either classical or quantum seiving/enumeration
           - output: return 1 if the model is classical model otherwise returns 0
           This function just is used so that we know if we are doing our search according to the 
           classical or quantum search
    """
    if reduction_option in [1,3]:
        return 1
    elif reduction_option in [2,4]:
        return 0
    else:
        return "invalid"

In [72]:
def estimate_security(N,d, lamda,p=3, reduction_option=1, fast_quick=1, cyclic=1, dec_pow=-1, twisted=False):
    """
    Check if the provided inputs provides the required level of security and return in this case
    N,d,q (that defines the cryptosystem) otherwise return -1 (indicating that the parameters do not provide the required level of security)
    Input: - N: the number of coefficients, d: the number of  1/-1/0 in the key
           - The decryption failure used here is 2^{-lamda}
           - reduction_option: refers to the reduction model to apply: default 1 refers to the classical sieving
             
           - fast_quick: if 1 means whenever the security evaluation at any point gives value lower than the required 
                 security level exit and don't go to the next estimation.
           - cyclic: refers if the underlying group is cyclic: the default calculations are with respect to the group
             
    Ouput: N,d,q the parameter for which the cryptosystem is defined to match lamda level of security.
           -1 otherwirse, when the input N,d can not define the required level of security.
    """
    #core_svp = get_core_svp(lamda)
    #r1 = get_r1(lamda)
  
        
    classical = is_it_classical(reduction_option)
    print("classical: ", classical)
    print("Security estimation for N ={}, d={}".format(N,d))           
    if fast_quick==1:
        print("Fast quick option is enabled!")
    if dec_pow==-1:
        dec_pow = lamda
    
    dec_failure = 2^-dec_pow
    dec_failure_pow = -dec_pow
    done = False
    k1 = get_combinatorial_search_log2(N,d,cyclic, classical)
    #print(int(comb))
    #k1 = round(math.log2(comb))
    print("k1: ", k1)
    if k1<lamda:
        print("combinatorial search failed!!")
        return -1
    
    sigma = np.sqrt(sigma_square(d, d,N, cyclic))  ## We are assuming that dr=df=d and the new design of ntru is considered and the cyclic group for sure
    #print("sigma: ", sigma)
    
    #This line I have commented to check the other way of reducing the value of q and d simultanously
#     q = get_q(dec_failure,sigma, N, old=0, binary=1, cyclic=cyclic)
#     print("binary q: ", q)
#     prime_q = get_q(dec_failure,sigma, N, old=0, binary=0, cyclic=cyclic)
#     print("prime q: ",prime_q)
 
#     (q,d,beta) = get_q_d(error_rate, sigma, N, old=0, binary=1, p, cyclic)
#     print("q: ", q)
#     print("d: ", d)

    while not done:
        #beta = get_beta_2016_estimator(N, d, q, p=3,cyclic=cyclic)
        (q,d,beta) = get_q_d(dec_failure, sigma, N, old=0, binary=1, p=p, cyclic=cyclic)
        print("q: ", q)
        print("d: ", d)
        print("beta: {} and the underlying group is: {}".format(beta,cyclic))
        arr, aux=final_security_estimation(N,q,d,lamda,beta, reduction_option, fast_quick,True,cyclic)
        print("array of security evaluation according to the models: ", arr)
        length = len(arr)
        k2 = arr[0]
        for i in range(1,length,1):
            k2= min(k2,arr[i]) ## The minimum of the security evaluations.
       
        if lamda> min(k2,k1) and fast_quick:
            print("The minimum cost of uSVP vs. hybird attack is: {}".format(k2))
            return -1
        q_prime = q/2
        dec_failure_prime = decryption_failure(N, (q_prime-2)/(2*p), sigma, cyclic)
        if dec_failure_prime<2^-lamda:
            q= q_prime
            dec_failure = dec_failure_prime
            dec_failure_pow = math.log2(dec_failure)##power of decryption failure
            print("updated q is {}".format(q)) ##updating the value of q
        else:
            done = True
            
    if twisted==True:
        N = int(N/2)
            
    return N,d,q,dec_failure_pow, [k1]+arr+[beta],aux
        

In [73]:
def estimate_security_no_dec_failure(N,d, lamda,p=3, reduction_option=1, fast_quick=1, cyclic=1, twisted=False):
    """
    This function finds the parameter of the scheme without considering decryption failure
    Totally correct parameter sets
    Check if the provided inputs provides the required level of security and return in this case
    N,d,q (that defines the cryptosystem) otherwise return -1 (indicating that the parameters do not provide the required level of security)
    Input: - N: the number of coefficients, d: the number of  1/-1/0 in the key
           - The decryption failure used here is 2^{-lamda}
           - reduction_option: refers to the reduction model to apply: default 1 refers to the classical sieving
             
           - fast_quick: if 1 means whenever the security evaluation at any point gives value lower than the required 
                 security level exit and don't go to the next estimation.
           - cyclic: refers if the underlying group is cyclic: the default calculations are with respect to the group(twisted group is treated as cyclic)
           - twisted: refers if the calculations are conducted for the twisted dihedral group ring
           
             
    Ouput: N,d,q the parameter for which the cryptosystem is defined to match lamda level of security.
           -1 otherwirse, when the input N,d can not define the required level of security.
    """
    #core_svp = get_core_svp(lamda)
    #r1 = get_r1(lamda)
    classical = is_it_classical(reduction_option)
    print("classical: ", classical)
    print("Security estimation for N ={}, d={}".format(N,d))           
    if fast_quick==1:
        print("Fast quick option is enabled!")
    
    done = False
    k1 = get_combinatorial_search_log2(N,d,cyclic, classical)
    #print(int(comb))
    #k1 = round(math.log2(comb))
    print("k1: ", k1)
    if k1<lamda:
        print("combinatorial search failed!!")
        return -1
    while not done:
        #beta = get_beta_2016_estimator(N, d, q, p=3,cyclic=cyclic)
        (q,d,beta) = get_q_d_without_error(N, binary=1, p=3, cyclic=1,old=1)
        print("q: ", q)
        print("d: ", d)
        print("beta: {} and the underlying group is: {}".format(beta,cyclic))
        arr, aux=final_security_estimation(N,q,d,lamda,beta, reduction_option, fast_quick,True,cyclic)
        print("array of security evaluation according to the models: ", arr)
        length = len(arr)
        k2 = arr[0]
        for i in range(1,length,1):
            k2= min(k2,arr[i]) ## The minimum of the security evaluations.
       
        if lamda> min(k2,k1) and fast_quick:
            print("The minimum cost of uSVP vs. hybird attack is: {}".format(k2))
            return -1
        done = True
    
    if twisted==True:
        N = int(N/2)
    return N,d,q, -math.inf, [k1]+arr+[beta],aux
        

In [74]:
## The dictionary save the parameter sets afte automatically generationg them
## the parameter sets can be interpreted as the follow:
##  N,d,q, the decryption failure, list contains the cost of the attacks against our cryptosystem [the combinatorial search, classical estimation(if specified), quantum estimation(if specified), both(if specified)]
#, another list contains the auxilarily infromation for the hybird attack; this information includes [k(places to search), r1(define the block to be reduced), beta(the blocksize needed for reduction)] 

## 1. Reproducing the parameters of  NTRU-HPS that achieve the first, third, and fifth levels of security
### Decryption failure is allowed

In [92]:
cyclic = 1 ##indicates the underlying group is cyclic (for twisted dihedral group, we treat the group as a cyclic group doesn't split)
           ## for 0 indicates the underlying group is the dihedral grouop that splits
level_sec_list = [128, 192, 256]
i =  -1
## core_svp = [125, 181, 254] ##The levels of the core-svp to consider according to the documentation of NTRUPrime
### For now, I will run the program according to the NTRU estimator and check if the results can be reproduced simialirly.
k_list =[128, 192, 256] ##The levels of security that we are trying to get: The levels of security are written in terms the Core-SVP
              ## metrics 
Dict = {}
classical_reduction_option = 1 ##classical-seiving
quantum_reduction_option   = 2 ##quantum-seiving
for index in range(len(k_list)):
    k = k_list[index]
    level_sec = level_sec_list[index]
    result = -1
    while result ==-1:
        
        i = i+1
        N = Ni[i] ###considering all the list of the possible primes
        d = int(N/3)
        print("checking for N = {}".format(N)) 
        result = estimate_security(N,d,k,3,classical_reduction_option,1,cyclic, level_sec)  ##classical only: we consider the pre-quantum Core-SVP
        print("res: ", result)
        print("---------------------------------------")
        
     
    Dict[str(k)+"_"+get_group_description(cyclic)+"_"+get_model_description(classical_reduction_option)] = result
    Dict[str(k)+"_"+get_group_description(cyclic)+"_"+get_model_description(quantum_reduction_option)] =  estimate_security(N,d,k,3,quantum_reduction_option,0,cyclic, level_sec)
    ### We apply the quantum reduction model without fast quick enabled so we can get the quantum cost that is less than lambda
    #print("classical result:", result) #The result is being understood as N,d,q,arr,aux
    
    

checking for N = 101
classical:  1
Security estimation for N =101, d=33
Fast quick option is enabled!
k1:  73
combinatorial search failed!!
res:  -1
---------------------------------------
checking for N = 103
classical:  1
Security estimation for N =103, d=34
Fast quick option is enabled!
k1:  74
combinatorial search failed!!
res:  -1
---------------------------------------
checking for N = 107
classical:  1
Security estimation for N =107, d=35
Fast quick option is enabled!
k1:  77
combinatorial search failed!!
res:  -1
---------------------------------------
checking for N = 131
classical:  1
Security estimation for N =131, d=43
Fast quick option is enabled!
k1:  96
combinatorial search failed!!
res:  -1
---------------------------------------
checking for N = 137
classical:  1
Security estimation for N =137, d=45
Fast quick option is enabled!
k1:  101
combinatorial search failed!!
res:  -1
---------------------------------------
checking for N = 139
classical:  1
Security estimation

KeyboardInterrupt: 

In [84]:
print("The parameter for NTRU \n")
print(Dict)

The parameter for NTRU 

{'128_cyclic_classical_seiving': (587, 195, 2048, -128, [455, 133.0, 128.0, 456.0], [166, 128, 438.0]), '128_cyclic_quantum_seiving': (587, 195, 2048, -128, [227, 121.0, 120.0, 456.0], [156, 128, 454.0]), '192_cyclic_classical_seiving': (863, 159, 2048, -192, [674, 205.0, 192.0, 701.0], [298, 192, 658.0]), '192_cyclic_quantum_seiving': (863, 159, 2048, -192, [337, 186.0, 181, 701.0], [282, 192, 684.0]), '256_cyclic_classical_seiving': (1109, 369, 4096, -256, [868, 261.0, 258.0, 893.0], [331, 256, 883.0]), '256_cyclic_quantum_seiving': (1109, 369, 4096, -256, [434, 237.0, 242.0, 893.0], [311, 256, 915.0])}


## 2. Reproducing the parameters of  DiTRU that achieve the first, third, and fifth levels of security
### Decryption failure is allowed

##### Takes longer time for the dihedral group
#### Uncomment the block below if you want to reproduce the parameters for DiTRU

In [1]:
# cyclic = 0 ##indicates the underlying group is cyclic (for twisted dihedral group, we treat the group as a cyclic group doesn't split)
#            ## for 0 indicates the underlying group is the dihedral grouop that splits
    
    
# ## For dihedral group it takes longer time compared to the parameter selection for the other schemes
# level_sec_list = [128, 192, 256]
# i =  -1
# ## core_svp = [125, 181, 254] ##The levels of the core-svp to consider according to the documentation of NTRUPrime
# ### For now, I will run the program according to the NTRU estimator and check if the results can be reproduced simialirly.
# k_list =[128, 192, 256] ##The levels of security that we are trying to get: The levels of security are written in terms the Core-SVP
#               ## metrics 
# Dict = {}
# classical_reduction_option = 1 ##classical-seiving
# quantum_reduction_option   = 2 ##quantum-seiving
# for index in range(len(k_list)):
#     k = k_list[index]
#     level_sec = level_sec_list[index]
#     result = -1
#     while result ==-1:
        
#         i = i+1
#         N = Ni[i] ###considering all the list of the possible primes, multiply by two for dihedral group
#         d = int(N/3)
#         print("checking for N = {}".format(N)) 
#         result = estimate_security(N,d,k,3,classical_reduction_option,1,cyclic, level_sec)  ##classical only: we consider the pre-quantum Core-SVP
#         print("res: ", result)
#         print("---------------------------------------")
        
#     Dict[str(k)+"_"+get_group_description(cyclic)+"_"+get_model_description(classical_reduction_option)] = result
#     Dict[str(k)+"_"+get_group_description(cyclic)+"_"+get_model_description(quantum_reduction_option)] =  estimate_security(N,d,k,3,quantum_reduction_option,0,cyclic, level_sec)
#     ### We apply the quantum reduction model without fast quick enabled so we can get the quantum cost that is less than lambda
#     #print("classical result:", result) #The result is being understood as N,d,q,arr,aux
    
    

In [ ]:
# print("The parameter for DiTRU \n")
# print(Dict)

## 3. Reproducing the parameters of  DiTRU$^{+}$ that achieve the first, third, and fifth levels of security
### Decryption failure is allowed

In [100]:
cyclic = 1 ##indicates the underlying group is cyclic (for twisted dihedral group, we treat the group as a cyclic group doesn't split)
           ## for 0 indicates the underlying group is the dihedral grouop that splits
level_sec_list = [128, 192, 192, 256, 256]
i =  -1
## core_svp = [125, 181, 254] ##The levels of the core-svp to consider according to the documentation of NTRUPrime
### For now, I will run the program according to the NTRU estimator and check if the results can be reproduced simialirly.
k_list =[128, 187, 192, 253, 256] ##The levels of security that we are trying to get: The levels of security are written in terms the Core-SVP
              ## metrics 
Dict = {}
classical_reduction_option = 1 ##classical-seiving
quantum_reduction_option   = 2 ##quantum-seiving
for index in range(len(k_list)):
    k = k_list[index]
    level_sec = level_sec_list[index]
    result = -1
    while result ==-1:
        
        i = i+1
        N = 2*Ni[i] ###considering all the list of the possible primes
        d = int(N/3)
        print("checking for N = {}".format(N)) 
        result = estimate_security(N,d,k,3,classical_reduction_option,1,cyclic, level_sec,True)  ##classical only: we consider the pre-quantum Core-SVP
        print("res: ", result)
        print("---------------------------------------")
        
     
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(classical_reduction_option)] = result
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(quantum_reduction_option)] =  estimate_security(N,d,k,3,quantum_reduction_option,0,cyclic, level_sec, True)
    ### We apply the quantum reduction model without fast quick enabled so we can get the quantum cost that is less than lambda
    #print("classical result:", result) #The result is being understood as N,d,q,arr,aux
    
    

checking for N = 202
classical:  1
Security estimation for N =202, d=67
Fast quick option is enabled!
k1:  152
beta initial before changing beta: 109.0
d after reducing q by two:  59
beta prev after reducing q by two: 125.0
q:  1024
d:  59
beta: 125.0 and the underlying group is: 1
array of security evaluation according to the models:  [36.0]
The minimum cost of uSVP vs. hybird attack is: 36.0
res:  -1
---------------------------------------
checking for N = 206
classical:  1
Security estimation for N =206, d=68
Fast quick option is enabled!
k1:  155
beta initial before changing beta: 112.0
d after reducing q by two:  59
beta prev after reducing q by two: 128.0
q:  1024
d:  59
beta: 128.0 and the underlying group is: 1
array of security evaluation according to the models:  [37.0]
The minimum cost of uSVP vs. hybird attack is: 37.0
res:  -1
---------------------------------------
checking for N = 214
classical:  1
Security estimation for N =214, d=71
Fast quick option is enabled!
k1:  1

In [101]:
print("The parameter for DiTRU^{+} \n")
print(Dict)

The parameter for DiTRU^{+} 

{'128_twisted_dihedral_classical_seiving': (293, 195, 2048, -128, [455, 133.0, 128.0, 456.0], [166, 128, 437.0]), '128_twisted_dihedral_quantum_seiving': (293, 195, 2048, -128, [227, 121.0, 120.0, 456.0], [156, 128, 453.0]), '187_twisted_dihedral_classical_seiving': (421, 159, 2048, -192, [657, 199.0, 187, 682.0], [287, 187, 642.0]), '187_twisted_dihedral_quantum_seiving': (421, 159, 2048, -192, [328, 181.0, 177.0, 682.0], [271, 187, 668.0]), '192_twisted_dihedral_classical_seiving': (443, 159, 2048, -192, [692, 211.0, 197, 723.0], [310, 192, 676.0]), '192_twisted_dihedral_quantum_seiving': (443, 159, 2048, -192, [346, 192.0, 186, 723.0], [293, 192, 703.0]), '253_twisted_dihedral_classical_seiving': (547, 364, 4096, -256, [856, 257.0, 253, 879.0], [326, 253, 868.0]), '253_twisted_dihedral_quantum_seiving': (547, 364, 4096, -256, [428, 233.0, 238.0, 879.0], [306, 253, 900.0]), '256_twisted_dihedral_classical_seiving': (557, 371, 4096, -256, [872, 262.0, 260

## 4. Reproducing the parameters of  DiTRU$^{\phi +}$ that achieve the first, third, and fifth levels of security
### Decryption failure is NOT allowed

In [ ]:
cyclic = 1 ##indicates the underlying group is cyclic for twisted dihedral group, we treat it as a cyclic
           ## for 0 indicates the dihedral group that splits
level_sec = [128, 192, 192, 256, 256 ]
i =  -1
## core_svp = [125, 181, 254] ##The levels of the core-svp to consider according to the documentation of NTRUPrime
### For now, I will run the program according to the NTRU estimator and check if the results can be reproduced simialirly.
k_list =[128, 182,192, 253, 256] ##The levels of security that we are trying to get: The levels of security are written in terms the Core-SVP
              ## metrics 
Dict = {}
classical_reduction_option = 1 ##classical-seiving
quantum_reduction_option   = 2 ##quantum-seiving
for k in k_list:
    result = -1
    while result ==-1:
        
        i = i+1
        N = 2*t[i]
        d = int(N/3)
        print("checking for N = {}".format(N)) 
        result = estimate_security_no_dec_failure(N,d,k,3,classical_reduction_option,1,cyclic, True)  ##classical only: we consider the pre-quantum Core-SVP
        print("res: ", result)
        print("---------------------------------------")
        
     
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(classical_reduction_option)] = result
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(quantum_reduction_option)] =  estimate_security_no_dec_failure(N,d,k,3,quantum_reduction_option,0,cyclic, True)
    ### We apply the quantum reduction model without fast quick enabled so we can get the quantum cost that is less than lambda
    #print("classical result:", result) #The result is being understood as N,d,q,arr,aux
    
    

checking for N = 202
classical:  1
Security estimation for N =202, d=67
Fast quick option is enabled!
k1:  152
d:  67
q:  2048
qprime:  1024
dprime:  63
q:  1024
d:  63
beta: 126.0 and the underlying group is: 1
array of security evaluation according to the models:  [37.0]
The minimum cost of uSVP vs. hybird attack is: 37.0
res:  -1
---------------------------------------
checking for N = 214
classical:  1
Security estimation for N =214, d=71
Fast quick option is enabled!
k1:  161
d:  71
q:  2048
qprime:  1024
dprime:  63
q:  1024
d:  63
beta: 136.0 and the underlying group is: 1
array of security evaluation according to the models:  [40.0]
The minimum cost of uSVP vs. hybird attack is: 40.0
res:  -1
---------------------------------------
checking for N = 262
classical:  1
Security estimation for N =262, d=87
Fast quick option is enabled!
k1:  199
d:  87
q:  2048
qprime:  1024
dprime:  63
q:  1024
d:  63
beta: 177.0 and the underlying group is: 1
array of security evaluation according

# No decryption failure

In [ ]:
print("The parameter for DiTRU^{\phi +} \n")
print(Dict)

## 4. Reproducing the parameters of  DiTRU$^{\phi +}$ that achieve levels of security similar to those introduced in the third round of NTRU
### Decryption failure is NOT allowed

In [ ]:
cyclic = 1 ##indicates the underlying group is cyclic for twisted dihedral group, we treat it as a cyclic
           ## for 0 indicates the dihedral group that splits
level_sec = [105, 144, 178]
i =  -1
## core_svp = [125, 181, 254] ##The levels of the core-svp to consider according to the documentation of NTRUPrime
### For now, I will run the program according to the NTRU estimator and check if the results can be reproduced simialirly.
k_list = [105, 144, 178] ##The levels of security that we are trying to get: The levels of security are written in terms the Core-SVP
              ## metrics 
Dict = {}
classical_reduction_option = 1 ##classical-seiving
quantum_reduction_option   = 2 ##quantum-seiving
for k in k_list:
    result = -1
    while result ==-1:
        
        i = i+1
        N = 2*t[i]
        d = int(N/3)
        print("checking for N = {}".format(N)) 
        result = estimate_security_no_dec_failure(N,d,k,3,classical_reduction_option,1,cyclic, True)  ##classical only: we consider the pre-quantum Core-SVP
        print("res: ", result)
        print("---------------------------------------")
        
     
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(classical_reduction_option)] = result
    Dict[str(k)+"_"+get_group_description(cyclic, True)+"_"+get_model_description(quantum_reduction_option)] =  estimate_security_no_dec_failure(N,d,k,3,quantum_reduction_option,0,cyclic, True)
    ### We apply the quantum reduction model without fast quick enabled so we can get the quantum cost that is less than lambda
    #print("classical result:", result) #The result is being understood as N,d,q,arr,aux
    
    

In [ ]:
print("The parameter for DiTRU^{\phi +} as per NIST third level of security \n")
print(Dict)